In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
import gc
import warnings
warnings.filterwarnings("ignore")



In [ ]:
# STEP 1: LOAD DATA
previous_application = pd.read_csv("/content/drive/My Drive/group6_Final_DP/data/raw_data/previous_application.csv")
previous_application = previous_application.sort_values(['SK_ID_CURR', 'DAYS_DECISION'])

# STEP 2: XỬ LÝ GIÁ TRỊ THIẾU
# Thay thế giá trị placeholder (365243.0) trong các cột thời gian
time_cols = ['DAYS_DECISION', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE', 'DAYS_TERMINATION']
for col in time_cols:
    previous_application[col] = previous_application[col].replace(365243.0, np.nan)

# Điền giá trị thiếu hợp lý
previous_application['AMT_ANNUITY'] = previous_application['AMT_ANNUITY'].fillna(0)
previous_application['AMT_APPLICATION'] = previous_application['AMT_APPLICATION'].fillna(0)
previous_application['AMT_CREDIT'] = previous_application['AMT_CREDIT'].fillna(0)
previous_application['AMT_DOWN_PAYMENT'] = previous_application['AMT_DOWN_PAYMENT'].fillna(0)
previous_application['RATE_DOWN_PAYMENT'] = previous_application['RATE_DOWN_PAYMENT'].fillna(0)
previous_application['CNT_PAYMENT'] = previous_application['CNT_PAYMENT'].fillna(1)
previous_application['SELLERPLACE_AREA'] = previous_application['SELLERPLACE_AREA'].fillna(0)

# STEP 3: LOẠI BỎ DỮ LIỆU TRÙNG LẶP
previous_application = previous_application.drop_duplicates()

# STEP 4: CHUẨN HÓA DỮ LIỆU SỐ
scaler = MinMaxScaler()
num_cols_to_scale = ['AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'SELLERPLACE_AREA']
previous_application[num_cols_to_scale] = scaler.fit_transform(previous_application[num_cols_to_scale])

# STEP 5: TẠO CÁC TÍNH NĂNG
# Tổng hợp số lần ứng dụng trước
pa_total_applications = previous_application.groupby('SK_ID_CURR')['SK_ID_PREV'].count()

# Tính tỷ lệ chấp thuận và từ chối
previous_application['IS_APPROVED'] = (previous_application['NAME_CONTRACT_STATUS'] == 'Approved').astype(int)
previous_application['IS_REFUSED'] = (previous_application['NAME_CONTRACT_STATUS'] == 'Refused').astype(int)
pa_approval_rate = previous_application.groupby('SK_ID_CURR')['IS_APPROVED'].mean()
pa_refusal_rate = previous_application.groupby('SK_ID_CURR')['IS_REFUSED'].mean()

# Tính khoảng thời gian giữa các lần ứng dụng
previous_application['application_interval'] = previous_application.groupby('SK_ID_CURR')['DAYS_DECISION'].diff(-1).fillna(0)
pa_avg_application_interval = previous_application.groupby('SK_ID_CURR')['application_interval'].mean()

# Tính tỷ lệ số tiền đã ứng dụng so với giá trị thực tế
previous_application['app_to_credit_ratio'] = previous_application['AMT_APPLICATION'] / (previous_application['AMT_CREDIT'] + 1e-9)
pa_avg_app_to_credit_ratio = previous_application.groupby('SK_ID_CURR')['app_to_credit_ratio'].mean()

# Tính số lượng hợp đồng có thời gian trả góp
pa_avg_payment_terms = previous_application.groupby('SK_ID_CURR')['CNT_PAYMENT'].mean()

# Tính giá trị trung bình và tổng các khoản vay đã chấp thuận
approved_apps = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Approved']
pa_total_approved_credit = approved_apps.groupby('SK_ID_CURR')['AMT_CREDIT'].sum()
pa_avg_approved_credit = approved_apps.groupby('SK_ID_CURR')['AMT_CREDIT'].mean()

# STEP 6: TÍCH HỢP DỮ LIỆU
pa_feature = pd.DataFrame({
    'pa_total_applications': pa_total_applications,
    'pa_approval_rate': pa_approval_rate,
    'pa_refusal_rate': pa_refusal_rate,
    'pa_avg_application_interval': pa_avg_application_interval,
    'pa_avg_app_to_credit_ratio': pa_avg_app_to_credit_ratio,
    'pa_avg_payment_terms': pa_avg_payment_terms,
    'pa_total_approved_credit': pa_total_approved_credit,
    'pa_avg_approved_credit': pa_avg_approved_credit
}).reset_index()

# Điền giá trị thiếu còn lại bằng 0
pa_feature = pa_feature.fillna(0)

# STEP 7: LƯU DỮ LIỆU
pa_feature.to_csv('/content/drive/My Drive/group6_Final_DP/data/processed_data/pa_feature.csv', index=False)
print("Previous Application features saved.")


Previous Application features saved.
